In [1]:
%load_ext ivortex
%vortex tmpcocoon

# [2017/08/18-19:43:09][vortex.sessions][_set_rundir:0147][WARNING]: Session <root> is changing its working directory </sauvegarde/vortex-dev/vortex/sphinx/training_dev_fr>
# [2017/08/18-19:43:09][vortex.sessions][_set_rundir:0150][INFO]: Session <root> set rundir </home/meunierlf/vortex-workdir/auto_cocoon_aDzBy4>


Vortex 1.1.4 loaded ( Friday 18. August 2017, at 19:43:08 )
The working directory is now: /home/meunierlf/vortex-workdir/auto_cocoon_aDzBy4/root


'/home/meunierlf/vortex-workdir/auto_cocoon_aDzBy4'

# Manipulation de ressources dans Vortex (découverte)

Une personne normalement constituée pourrait décrire une demande de la façon suivante :
> Données en point de grille à l'échéance 3h (sur le domaine EUROC25, au 
> format GRIB) pour le modèle Arpège du réseau de production du 01/06/2017 18UTC
> pour le second membre de la PEARP en chaîne en double (récupération 
> via le système d'archivage).

Une idée clé de Vortex est de conserver toute la richesse d'une telle description.

Curieusement, dans la plupart des scripts, on trouve cela :

``ftp://hendrix.meteo.fr/~mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib``

## Objets Vortex de description des données

Avec Vortex, cette approche descriptive se traduit par l'association de trois objets :
  * Un objet de classe de base **vortex.data.resources.Resource**. Cet objet décrit la donnée en elle-même. Dans l'exemple précédent : "Données en point de grille à l'échéance 3h (sur le domaine EUROC25, au format GRIB) pour le modèle Arpège du réseau de production du 01/06/2017 18UTC".
  * Un objet de classe de base **vortex.data.providers.Provider**. Cet objet décrit la façon dont sera récupérée la donnée. Dans l'exemple précédent : "Second membre de la PEARP en chaîne en double (récupération via le système d'archivage)".
  * Un objet de classe de base **vortex.data.containers.Container**. Cet objet décrit l'endroit où sera stockée la ressource localement. Il s'agit quasi-systématiquement d'un fichier (décrit par son chemin dans le système de fichiers).

En vortex cela donne... Pour l'objet *resource* :

In [2]:
pe_resource = fp.proxy.resource(kind='gridpoint',
                                term=3,
                                geometry='euroc25',
                                nativefmt='grib',
                                model='arpege',
                                cutoff='production',
                                date='2017060118',
                                origin='historic')
print pe_resource

<common.data.gridfiles.GridPointExport object at 0x7fc99860a1d0 | origin='hst' cutoff='production' term='03:00' geometry='<vortex.data.geometries.LonlatGeometry | tag='euroc25' id='Large european BDAP target' area='EUROC25' r='02dg500'>' date='2017-06-01T18:00:00Z' model='arpege'>


Pour l'objet *provider* :

In [3]:
pe_provider = fp.proxy.provider(vapp='arpege',
                                vconf='pearp',
                                member=2,
                                experiment='DBLE',
                                namespace='vortex.archive.fr',
                                block='forecast')
print pe_provider

<vortex.data.providers.VortexOp object at 0x7fc9985ba050 | namespace='vortex.archive.fr' block='forecast'>


Pour l'objet *container* :

In [4]:
pe_container = fp.proxy.container(local='my_pearp_gribfile', format='grib')
print pe_container

<vortex.data.containers.SingleFile object at 0x7fc9897cb250 | path='my_pearp_gribfile'>


## Faire interagir ces objets : le *resource's Handler*

Pour manipuler des données dans Vortex, il faut associer les trois objets évoqués précédemment. Cette agrégation est faite par un objet de classe **vortex.data.handlers.Handler**.

Cette objet permet la manipulation des données via des méthodes simples :
  * **locate** : emplacement physique des données (ici l'adresse ftp) ;
  * **get** : récupérer la donnée et la placer dans le *container* ;
  * **put** : envoyer le contenu du *container* à l'emplacement décrit par les objets *resource* et *provider* ;
  * **check** : vérifier l'existence de la donnée ;
  * **delete** : effacer la donnée décrite par les objets *resource* et *provider* ;
  * **clear** : supprimer le contenu du *container*

En pratique...

In [5]:
rhandler = vortex.data.handlers.Handler(dict(resource=pe_resource,
                                             provider=pe_provider,
                                             container=pe_container))

Récupération des données :

In [6]:
rhandler.get()
%ls -l

# [2017/08/18-19:43:09][vortex.data.stores][ftpget:0997][INFO]: ftpget on ftp://hendrix.meteo.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: my_pearp_gribfile)
# [2017/08/18-19:43:09][vortex.tools.net][get:0321][INFO]: FTP <get:/home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib>


total 1328
-rw-r--r-- 1 meunierlf algo 1357578 août  18 19:43 my_pearp_gribfile


Un peu de ménage pour la suite...

In [7]:
rhandler.clear()

True

In [8]:
%ls -l

total 0


## Usage des données

La classe **Handler** ne répond pas vraiment à tous les besoins, on aimerait pouvoir préciser :
  * Ce que l'on veut faire des données
  * Ce que l'on doit faire si une erreur se produit
  * Si la donnée doit être modifiée ou non

C'est le rôle de l'objet de classe **vortex.layout.dataflow.Section** qui enrobe le *resource's Handler* :

In [9]:
section = vortex.layout.dataflow.Section(rh=rhandler,
                                         # Ce fichier fait office de :
                                         role='InitialCondition',
                                         # Il s'agit d'un fichier d'entrée :
                                         kind=vortex.layout.dataflow.ixo.INPUT,
                                         # Plantage sur erreur :
                                         fatal=True,
                                         # Lecture/Ecriture :
                                         intent='inout')

Cet objet a deux méthodes principales **get** et **put** qui se chargent d'appeler le *resource's Handler* en tenant compte des attributs *fatal* et *intent* :

In [10]:
section.get()
%ls -l

# [2017/08/18-19:43:10][vortex.data.stores][ftpget:0997][INFO]: ftpget on ftp://hendrix.meteo.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: my_pearp_gribfile)
# [2017/08/18-19:43:12][vortex.tools.net][get:0321][INFO]: FTP <get:/home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib>


total 1328
-rw-r--r-- 1 meunierlf algo 1357578 août  18 19:43 my_pearp_gribfile


Le *resource's Handler* reste accessible (par exemple, pour nettoyer le *container*) :

In [11]:
section.rh.clear()
%ls -l

total 0


## Schéma récapitulatif

![](images/data_management_overview.png)

## Création rapide de *Handlers* et de sections via le module Toolbox

Il est fastidieux de créer les différents objets manuellement, le module **vortex.toolbox** (souvent systématiquement importé) met donc à disposition des fonctions permettant la création d'ensemble d'objets en un seul appel.

### Création rapide de *resource's Handlers*

In [12]:
rhandlers = toolbox.rload(# Resource
                          kind='gridpoint', term=3, geometry='euroc25',
                          nativefmt='grib', model='arpege', 
                          cutoff='production', date='2017060118',
                          origin='historic',
                          # Provider
                          vapp='arpege', vconf='pearp', member=2,
                          experiment='DBLE', namespace='vortex.archive.fr',
                          block='forecast',
                          # Container
                          local='my_pearp_gribfile', format='grib')
print rhandlers

On obtient une liste de *resource's Handlers* et non un objet unique... nous reviendrons sur cela plus tard ; dans l'immédiat, acceptons cet état de fait. L'objet contenu dans la liste est bien celui attendu :

In [13]:
rhandlers[0].quickview()

00. <vortex.data.handlers.Handler object at 0x7fc98977ae10>
  Complete  : True
  Container : <vortex.data.containers.SingleFile object at 0x7fc98977ae90 | path='my_pearp_gribfile'>
  Provider  : <vortex.data.providers.VortexOp object at 0x7fc989797b10 | namespace='vortex.archive.fr' block='forecast'>
  Resource  : <common.data.gridfiles.GridPointExport object at 0x7fc9986639d0 | origin='hst' cutoff='production' term='03:00' geometry='<vortex.data.geometries.LonlatGeometry | tag='euroc25' id='Large european BDAP target' area='EUROC25' r='02dg500'>' date='2017-06-01T18:00:00Z' model='arpege'>


### Création rapide de sections

In [14]:
rhandlers = toolbox.input(# Input function options
                          verbose=False,
                          # Section
                          role='InitialCondition', fatal=True, intent='inout',
                          # Resource
                          kind='gridpoint', term=3, geometry='euroc25',
                          nativefmt='grib', model='arpege', 
                          cutoff='production', date='2017060118',
                          origin='historic',
                          # Provider
                          vapp='arpege', vconf='pearp', member=2,
                          experiment='DBLE', namespace='vortex.archive.fr',
                          block='forecast',
                          # Container
                          local='my_pearp_gribfile', format='grib')
print rhandlers

* Là aussi, on obtient une liste de *resource's Handlers*. Cela peut paraître étrange mais les objets *Section* associés existent bel et bien (ils sont stockés et peuvent être récupérés).
* La fonction **input** accepte quelques options :
    * **verbose** : Si *verbose=True* (le défaut), un résumé est affiché sur la sortie standard ;
    * **loglevel** : Le niveau de log souhaité pour Vortex ('info' par défaut) ;
    * **now** : Si *now=True* (*False* par défaut), la ressource est récupérée sur le champ par un appel à
      la méthode *get*. C'est souvent ce que l'on souhaite ;
    * **insitu** : Si *insitu=True*, qu'un *resource's Handler* identique a déjà récupéré la données
      lors d'une précédente exécution et que le *container* est rempli, on se contente de créer
      les objets *Section* et *Handler* (on ne re-récupère pas la donnée).
* Il existe aussi une méthode **toolbox.output** utilisée pour la création de *Sections* destinées aux
  fichiers de sorties (dans ce cas *now=True*, implique un appel immédiat à la méthode *put*).

In [15]:
rhandlers = toolbox.input(# Input function options
                          verbose=False, now=True,
                          # Section
                          role='InitialCondition', fatal=True, intent='inout',
                          # Resource
                          kind='gridpoint', term=3, geometry='euroc25',
                          nativefmt='grib', model='arpege', 
                          cutoff='production', date='2017060118',
                          origin='historic',
                          # Provider
                          vapp='arpege', vconf='pearp', member=2,
                          experiment='DBLE', namespace='vortex.archive.fr',
                          block='forecast',
                          # Container
                          local='my_pearp_gribfile', format='grib')
print rhandlers
rhandlers[0].clear()  # Nettoyage

# [2017/08/18-19:43:12][vortex.data.stores][ftpget:0997][INFO]: ftpget on ftp://hendrix.meteo.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: my_pearp_gribfile)
# [2017/08/18-19:43:13][vortex.tools.net][get:0321][INFO]: FTP <get:/home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib>


True

Il est possible de consulter et modifier certains défauts utilisés par le module toolbox :

In [16]:
toolbox.show_toolbox_settings()

+ active_now               = False
+ active_insitu            = False
+ active_verbose           = True
+ active_promise           = True
+ active_clear             = False
+ active_metadatacheck     = True
+ active_incache           = False


In [17]:
toolbox.active_now = True

La récupération des données se fait désormais automatiquement même si `now=True` est omis :

In [18]:
rhandlers = toolbox.input(# Input function options
                          verbose=False,
                          # Section
                          role='InitialCondition', fatal=True, intent='inout',
                          # Resource
                          kind='gridpoint', term=3, geometry='euroc25',
                          nativefmt='grib', model='arpege', 
                          cutoff='production', date='2017060118',
                          origin='historic',
                          # Provider
                          vapp='arpege', vconf='pearp', member=2,
                          experiment='DBLE', namespace='vortex.archive.fr',
                          block='forecast',
                          # Container
                          local='my_pearp_gribfile', format='grib')
print rhandlers
rhandlers[0].clear()  # Nettoyage

# [2017/08/18-19:43:14][vortex.data.stores][ftpget:0997][INFO]: ftpget on ftp://hendrix.meteo.fr//home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib (to: my_pearp_gribfile)
# [2017/08/18-19:43:14][vortex.tools.net][get:0321][INFO]: FTP <get:/home/m/mxpt/mxpt001/vortex/arpege/pearp/DBLE/2017/06/01/T1800P/mb002/forecast/grid.arpege-forecast.euroc25+0003:00.grib>


True

## Conclusion intermédaire

* Via le module *toolbox*, il est assez simple, dans un script, une session Python interactive ou un notebook, de créer tous les objets nécessaires à la manipulation de ressources.
* Reste que les objets *Resource*, *Provider* et *Container* sont créés de façon assez mystérieuse (en fonction des arguments spécifiés par l'utilisateur). Tous ces objets ont pourtant des types bien précis :

In [19]:
print 'Resource:  ', type(rhandlers[0].resource)
print 'Provider:  ', type(rhandlers[0].provider)
print 'Container: ', type(rhandlers[0].container)

Resource:   <class 'common.data.gridfiles.GridPointExport'>
Provider:   <class 'vortex.data.providers.VortexOp'>
Container:  <class 'vortex.data.containers.SingleFile'>


> C'est tout l'objet de la présentation suivante...